# **Analyzing and Predicting Business Activity for Cafes and Restaurants in Melbourne**

**Authored by:** Sachitha Sadeepa Kasthuriarachchi

**Duration:** TBD

**Level:** Beginner/Intermediate

**Pre-requisite Skills:**  Python Programming,Jupyter Notebooks,Power BI,Data Analysis,Machine Learning and Geospatial Analysis.


**Background:**

I am planning to open a new cafe in Melbourne and want to ensure that my business meets customer demands and adheres to area-specific trends. To help with this, I am using a comprehensive dataset of existing cafes and restaurants, which includes information about their seating capacity, location, industry type, and other relevant factors.

**Objectives:**

1. Distribution Analysis: Understand the distribution of cafes and restaurants across different areas of Melbourne.

2. Seating Capacity Analysis: Compare the seating capacity for indoor and outdoor seating.
   
3. Industry Analysis: Explore the prevalence of different industries within the cafes and restaurants sector.
     
4. Geospatial Analysis: Visualize the geographic distribution of cafes and restaurants across Melbourne.
    
5. Trend Analysis: Identify trends in business activity over different census years.

6. Prediction Task: Develop a predictive model to forecast the number of seats based on various features.

**Scenario:**

1. **Informed Decision-Making:** I use the insights provided by the data analysis to make data-driven decisions about my new cafe, reducing the risk of underestimating or overestimating seating capacity.

2. **ompetitive Advantage:** By understanding local trends and customer preferences, I can tailor my business strategy to stand out in a competitive market.

3. **Resource Optimization:** The predictive model helps me allocate resources effectively, ensuring I invest appropriately in indoor and outdoor seating based on demand forecasts.

## 1. Data Loading and Examination

Below are the links to the four data sets that will be used in this for this use case.

[Data Set 1](https://data.melbourne.vic.gov.au/explore/dataset/cafes-and-restaurants-with-seating-capacity/information/) **Café, restaurant, bistro seats.** This dataset contains **60055** records.

[Data Set 2](https://data.melbourne.vic.gov.au/explore/dataset/blocks-for-census-of-land-use-and-employment-clue/information/?location=13,-37.80246,144.94417&basemap=mbs-7a7333) **Blocks for Census of Land Use and Employment (CLUE).** This dataset contains **606** records.


[Data Set 3](https://data.melbourne.vic.gov.au/explore/dataset/employment-by-block-by-clue-industry/information/) **Jobs per CLUE industry for blocks.** This dataset contains **12349** records.


[Data Set 4](https://data.melbourne.vic.gov.au/explore/dataset/floor-space-by-use-by-block/information/) **Floor space per space use for blocks.** This dataset contains **12349** records.


### 1.0 Dataset Imported through API

In [12]:
import requests
import pandas as pd
from io import StringIO

#Function to collect data
def collect_data(dataset_id):
    base_url = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
    #apikey = api_key #use if use datasets API_key permissions
    dataset_id = dataset_id
    format = 'csv'

    url = f'{base_url}{dataset_id}/exports/{format}'
    params = {
        'select': '*',
        'limit': -1,  # all records
        'lang': 'en',
        'timezone': 'UTC',
        #'api_key': apikey  #use if use datasets API_key permissions
    }

    # GET request
    response = requests.get(url, params=params)

    if response.status_code == 200:
        # StringIO to read the CSV data
        url_content = response.content.decode('utf-8')
        dataset = pd.read_csv(StringIO(url_content), delimiter=';')
        return dataset
    else:
        print(f'Request failed with status code {response.status_code}')



### 1.2 Load the Dataset

In [13]:
# Set dataset_id to query for the API call dataset name
dataset_1_id = 'cafes-and-restaurants-with-seating-capacity'
dataset_2_id = 'blocks-for-census-of-land-use-and-employment-clue'
dataset_3_id = 'employment-by-block-by-clue-industry'
dataset_4_id = 'floor-space-by-use-by-block'

In [14]:
# Save dataset to df1.df2.df3 and df4 varaible
df1 = collect_data(dataset_1_id)
df2 = collect_data(dataset_2_id)
df3 = collect_data(dataset_3_id)
df4 = collect_data(dataset_4_id)

### 1.3 Exmine the Datasets

#### 1.3.1 Data Set 1

In [15]:
# Check number of records in df1
print(f'The dataset contains {len(df1)} records.')
# View df1
df1.head(3)

The dataset contains 60055 records.


,census_year,block_id,property_id,base_property_id,building_address,clue_small_area,trading_name,business_address,industry_anzsic4_code,industry_anzsic4_description,seating_type,number_of_seats,longitude,latitude,location
0,2017,6,578324,573333,2 Swanston Street MELBOURNE 3000,Melbourne (CBD),Transport Hotel,"Tenancy 29, Ground , 2 Swanston Street MELBOUR...",4520,"Pubs, Taverns and Bars",Seats - Indoor,230,144.969942,-37.817778,"-37.817777826050005, 144.96994164279243"
1,2017,6,578324,573333,2 Swanston Street MELBOURNE 3000,Melbourne (CBD),Transport Hotel,"Tenancy 29, Ground , 2 Swanston Street MELBOUR...",4520,"Pubs, Taverns and Bars",Seats - Outdoor,120,144.969942,-37.817778,"-37.817777826050005, 144.96994164279243"
2,2017,11,103957,103957,517-537 Flinders Lane MELBOURNE 3000,Melbourne (CBD),Altius Coffee Brewers,"Shop , Ground , 517 Flinders Lane MELBOURNE 3000",4512,Takeaway Food Services,Seats - Outdoor,4,144.956486,-37.819875,"-37.819875445799994, 144.95648638781466"


#### 1.3.1 Data Set 2

In [16]:
# Check number of records in df2
print(f'The dataset contains {len(df2)} records.')
# View df2
df2.head(3)

The dataset contains 606 records.


,geo_point_2d,geo_shape,block_id,clue_area
0,"-37.82296169692379, 144.95049282288122","{""coordinates"": [[[144.9479230372, -37.8233694...",1112,Docklands
1,"-37.78537422996195, 144.94085920366408","{""coordinates"": [[[144.9426153438, -37.7866287...",927,Parkville
2,"-37.777687358375964, 144.94600024715058","{""coordinates"": [[[144.9425926939, -37.7787229...",929,Parkville


#### 1.3.1 Data Set 3

In [17]:
# Check number of records in df3
print(f'The dataset contains {len(df3)} records.')
# View df3
df3.head(3)

The dataset contains 12394 records.


,census_year,block_id,clue_small_area,accommodation,admin_and_support_services,agriculture_and_mining,arts_and_recreation_services,business_services,construction,education_and_training,...,information_media_and_telecommunications,manufacturing,other_services,public_administration_and_safety,real_estate_services,rental_and_hiring_services,retail_trade,transport_postal_and_storage,wholesale_trade,total_jobs_in_block
0,2022,4,Melbourne (CBD),0.0,0.0,0.0,362.0,0.0,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,38.0,368.0,0.0,1008.0
1,2022,5,Melbourne (CBD),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022,6,Melbourne (CBD),0.0,0.0,0.0,203.0,0.0,0.0,0.0,...,NaN,0.0,NaN,0.0,NaN,0.0,47.0,0.0,0.0,647.0


#### 1.3.1 Data Set 4

In [18]:
# Check number of records in df4
print(f'The dataset contains {len(df4)} records.')
# View df4
df4.head(3)

The dataset contains 12394 records.


,census_year,block_id,clue_small_area,commercial_accommodation,common_area,community_use,educational_research,entertainment_recreation_indoor,equipment_installation,hospital_clinic,...,transport,transport_storage_uncovered,unoccupied_under_construction,unoccupied_under_demolition_condemned,unoccupied_under_renovation,unoccupied_undeveloped_site,unoccupied_unused,wholesale,workshop_studio,total_floor_space_in_block
0,2013,2387,North Melbourne,0.0,NaN,0.0,NaN,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,44471.0
1,2013,2390,North Melbourne,0.0,1040.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5357.0,NaN,5071.0,48332.0
2,2013,2501,Kensington,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21456.0


## 2.0 Data Cleaning and Pre processing Data

***

_**DELETE BEFORE PUBLISHING**_

## Style guide for use cases

### Headers

For styling within your markdown cells, there are two choices you can use for headers.

1) You can use HTML classes specific to the use case styling:

```<p class="usecase-subsection-header">This is a subsection header.</p>```

<p style="font-weight: bold; font-size: 1.2em;">This is a subsection header.</p>

```<p class="usecase-subsection-blurb">This is a blurb header.</p>```

<p style="font-weight: bold; font-size: 1em; font-style:italic;">This is a blurb header.</p>


2) Or if you like you can use the markdown header styles:

```# for h1```

```## for h2```

```### for h3```

```#### for h4```

```##### for h5```

## Plot colour schemes

General advice:
1. Use the same colour or colour palette throughout your notebook, unless variety is necessary
2. Select a palette based on the type of data being represented
3. Consider accessibility (colourblindness, low vision)

#### 1) If all of your plots only use 1-2 colors use one of the company style colors:

| Light theme | Dark Theme |
|-----|-----|
|<p style="color:#2af598;">#2af598</p>|<p style="color:#08af64;">#08af64</p>|
|<p style="color:#22e4ac;">#22e4ac</p>|<p style="color:#14a38e;">#14a38e</p>|
|<p style="color:#1bd7bb;">#1bd7bb</p>|<p style="color:#0f9295;">#0f9295</p>|
|<p style="color:#14c9cb;">#14c9cb</p>|<p style="color:#056b8a;">#056b8a</p>|
|<p style="color:#0fbed8;">#0fbed8</p>|<p style="color:#121212;">#121212</p>|
|<p style="color:#08b3e5;">#08b3e5</p>||


#### 2) If your plot needs multiple colors, choose an appropriate palette using either of the following tutorials:
- https://seaborn.pydata.org/tutorial/color_palettes.html
- https://matplotlib.org/stable/tutorials/colors/colormaps.html

#### 3) Consider accessibility as well.

For qualitative plotting Seaborn's 'colorblind' palette is recommended. For maps with sequential or diverging it is recommended to use one of the Color Brewer schemes which can be previewed at https://colorbrewer2.org/.

If you want to design your own colour scheme, it should use the same principles as Cynthia Brewer's research (with variation not only in hue but also, saturation or luminance).

### References

Be sure to acknowledge your sources and any attributions using links or a reference list.

If you have quite a few references, you might wish to have a dedicated section for references at the end of your document, linked using footnote style numbers.

You can connect your in-text reference by adding the number with a HTML link: ```<a href="#fn-1">[1]</a>```

and add a matching ID in the reference list using the ```<fn>``` tag: ```<fn id="fn-1">[1] Author (Year) _Title_, Publisher, Publication location.</fn>```